In [19]:
import os

In [20]:
%pwd

'/Users/amoghagadde/Desktop/Amogha/Projects/Data_Science/Text-Summarizer'

In [21]:
os.chdir("../")

In [22]:
%pwd

'/Users/amoghagadde/Desktop/Amogha/Projects/Data_Science'

In [23]:
# import torch
# if torch.backends.mps.is_available():
#     mps_device = torch.device("mps")
#     x = torch.ones(1, device=mps_device)
#     print (x)
# else:
#     print ("MPS device not found.")

In [24]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


In [25]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [26]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [27]:
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
# device = torch.device("cpu")

In [28]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [29]:
# class ModelTrainer:
#     def __init__(self, config: ModelTrainerConfig):
#         self.config = config


    
#     def train(self):
#         # device = "cuda" if torch.cuda.is_available() else "cpu"
#         device = torch.device("cpu")
#         print(f"Using device: {device}")
#         # device = torch.device('mps')
#                 # Check that MPS is available
#         # if not torch.backends.mps.is_available():
#         #     if not torch.backends.mps.is_built():
#         #         print("MPS not available because the current PyTorch install was not "
#         #               "built with MPS enabled.")
#         #     else:
#         #         print("MPS not available because the current MacOS version is not 12.3+ "
#         #               "and/or you do not have an MPS-enabled device on this machine.")
#         #     device = torch.device("cpu")
#         # else:
#         #     device = torch.device("mps")

#         tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
#         # model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
#         model_t5 = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
#         seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_t5)
        
#         #loading data 
#         dataset_samsum_pt = load_from_disk(self.config.data_path)

#         # trainer_args = TrainingArguments(
#         #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
#         #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
#         #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
#         #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
#         #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
#         # ) 


#         trainer_args = TrainingArguments(
#             output_dir=self.config.root_dir, 
#             num_train_epochs=1, 
#             warmup_steps=500,
#             per_device_train_batch_size=16, #1, 
#             per_device_eval_batch_size=16, #1,
#             weight_decay=0.01, 
#             logging_steps=50,
#             evaluation_strategy='steps', 
#             eval_steps=500, 
#             save_steps=500, #1e6,
#             gradient_accumulation_steps=2, #16, 
#             fp16=False
#         ) 

#         trainer = Trainer(model=model_t5, args=trainer_args,
#                   tokenizer=tokenizer, data_collator=seq2seq_data_collator,
#                   train_dataset=dataset_samsum_pt["train"], 
#                   eval_dataset=dataset_samsum_pt["validation"])
        
#         trainer.train()

#         ## Save model
#         model_t5.save_pretrained(os.path.join(self.config.root_dir,"t5-samsum-model"))
#         ## Save tokenizer
#         tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [30]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = torch.device("cpu")
        print(f"Using device: {device}")
        
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_t5 = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_t5)
        
        # Loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        # Sampling a smaller subset of the dataset
        train_subset = dataset_samsum_pt["train"].select(range(1000))  # First 1000 samples
        eval_subset = dataset_samsum_pt["validation"].select(range(200))  # First 200 samples
        
        
        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, 
            num_train_epochs=1, 
            warmup_steps=500,
            per_device_train_batch_size=8,  
            per_device_eval_batch_size=8, 
            weight_decay=0.01, 
            logging_steps=50,
            evaluation_strategy='steps', 
            eval_steps=500, 
            save_steps=500, 
            gradient_accumulation_steps=4, 
            fp16=False
        ) 

        trainer = Trainer(model=model_t5, args=trainer_args,
                          tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                          train_dataset=train_subset, 
                          eval_dataset=eval_subset)
        
        trainer.train()

        ## Save model
        model_t5.save_pretrained(os.path.join(self.config.root_dir,"t5-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))


In [31]:
# import torch

# print("CUDA available:", torch.cuda.is_available())
# print("MPS available:", torch.backends.mps.is_available())
# print("Current device:", torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")))

In [32]:
# import os

# print("PYTORCH_ENABLE_MPS_FALLBACK:", os.getenv("PYTORCH_ENABLE_MPS_FALLBACK"))
# print("PYTORCH_MPS_HIGH_WATERMARK_RATIO:", os.getenv("PYTORCH_MPS_HIGH_WATERMARK_RATIO"))
# print("CUDA_VISIBLE_DEVICES:", os.getenv("CUDA_VISIBLE_DEVICES"))

In [33]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.4.0


In [34]:
# import torch
# torch.cuda.empty_cache()

In [35]:
# print("CUDA available:", torch.cuda.is_available())
# print("MPS available:", torch.backends.mps.is_available())
# print("Current device:", torch.cuda.current_device() if torch.cuda.is_available() else "CPU")

In [36]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-08-08 23:28:55,664: INFO: common: yaml file: /Users/amoghagadde/Desktop/Amogha/Projects/Data_Science/Text-Summarizer/config/config.yaml loaded successfully]
[2024-08-08 23:28:55,665: INFO: common: yaml file: /Users/amoghagadde/Desktop/Amogha/Projects/Data_Science/Text-Summarizer/params.yaml loaded successfully]
[2024-08-08 23:28:55,666: INFO: common: created directory at: artifacts]
[2024-08-08 23:28:55,666: INFO: common: created directory at: /Users/amoghagadde/Desktop/Amogha/Projects/Data_Science/Text-Summarizer/artifacts/model_trainer]
Using device: cpu


/Users/amoghagadde/Desktop/Amogha/Jupyter/anaconda3/envs/textS/lib/python3.8/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/31 [00:00<?, ?it/s]

{'train_runtime': 766.7893, 'train_samples_per_second': 1.304, 'train_steps_per_second': 0.04, 'train_loss': 7.89677478421119, 'epoch': 0.99}


In [37]:
# print("CUDA available:", torch.cuda.is_available())
# print("MPS available:", torch.backends.mps.is_available())
# print("Current device:", torch.cuda.current_device() if torch.cuda.is_available() else "CPU")